# Predator Prey Model

by Sri Vallabha, Ph.D <br>
Indian Institute of Science, Bangalore, India <br>

<h3>Code Revisions and Modifications for Interaction</h3>

by L. Van Warren <br>
University of Arkansas, Little Rock, Arkansas, USA

Licensed under CC BY-NC 3.0.

<pre>
# Optional: To change style of notebook
# from IPython.core.display import HTML
# HTML(open('prey.css', "r").read())
</pre>

Source |
[GIT Original Source](https://goo.gl/pnoCA8) |
[Markdown Quick Reference](https://goo.gl/yycqtU) |
[Markdown Indentation](https://goo.gl/XVKyYe) |
[LaTex Font Sizes](https://goo.gl/q9hLnx) |
[Lotka-Volterra Equations](https://goo.gl/rfNB4N) |

### Prey-Predator System as a pair of first-order, non-linear, differential equations
Prey Equation Assumptions
>  Prey (x) have unlimited food <br>
>   αx   prey population increase growth rate <br>
>  βxy prey population decrease growth rate
    
We write:

>$\Large\frac{dx}{dt}\normalsize = \alpha x - \beta x y $</indent>

Predator Equation Assumptions:
> Predators (y) have limited food Prey (x) <br>
>   ɣy predator population decrease growth rate <br>
>  ẟxy predator population increase growth rate
    
We write:

>$\Large\frac{dy}{dt}\normalsize = \delta x y -\gamma y $

<h4>Euler forward time stepping scheme:</h4>

<b>euler_step(u, f, dt)</b>:
returns the solution at next time-step using Euler's method.<br>
Inputs
> u      : array of floats containing solution at the previous time-step.<br>
> f      : function to compute the RHS of the system of equation.<br>
> dt     : scalar float time-increment.<br>

Outputs
> u[n+1] : array of floats approximate solution at the next time step.


<h4>Lotka-Volterra function:</h4>

<b>f(u)</b>:
returns rate of change of Prey (x) and Predator (y).<br>
Inputs
> u : array of floats containing solution at time n.<br>

Outputs

> $\Large\frac{du}{dt}\normalsize$ : array of floats containing the RHS given u.

In [1]:
import math                      # for mathematical operations (like math.h)
import matplotlib.pyplot as plt  # for plotting
import numpy as np               # for numerical operations
from ipywidgets import *         # for interactive animation
                                 # forces graph appearance in jupyter notebooks
%matplotlib inline      

In [92]:
plt.rcParams["figure.figsize"] = [16, 10]

In [93]:
# Set initial parameters
α = 1.0
β = 1.2
ɣ = 4.0
ẟ = 1.0

In [94]:
# Time stepping scheme - euler forward
def euler_step(u, f, α, β, ɣ, ẟ, dt):
    return u + dt * f(u, α, β, ɣ, ẟ)

In [95]:
# Lotka-Volterra function
def lotka(u, α, β, ɣ, ẟ):
    x = u[0]
    y = u[1]
    return np.array([x*(α - β*y), -y*(ɣ - ẟ*x)])

In [96]:
def solve_w_euler(α, β, ɣ, ẟ):
    T  = 15.0                           # final time
    dt =  0.01                          # set time-increment
    x0 = 10.0
    y0 =  2.0
    t0 =  0.0

    N  = int(T/dt) + 1                  # number of time-steps

    # set initial conditions
    u_euler = np.empty((N, 2))

    # initialize the array containing the solution for each time-step
    u_euler[0] = np.array([x0, y0])

    # use a for loop to call the function rk2_step()
    for n in range(N-1):
        u_euler[n+1] = euler_step(u_euler[n], lotka, α, β, ɣ, ẟ, dt)
    
    time = np.linspace(0.0, T,N)
    x = u_euler[:,0]
    y = u_euler[:,1]
    
    return x, y, time

We will now plot the variation of population for each species with time.

Source |
[Legend Location](https://goo.gl/K86yBa) |
[LaTex Font Sizes](https://goo.gl/q9hLnx) |
[Lotka-Volterra Equations](https://goo.gl/rfNB4N) |

In [118]:
def plot_pp_v_t(foo, x, y, time):
    foo.set_xlabel("time", fontsize=14)
    foo.set_ylabel("Population", fontsize=14)
    foo.set_title("Predator-Prey Population", fontsize=16)
    foo.plot(time, x, label = 'prey ')
    foo.plot(time, y, label = 'predator')
    foo.legend(loc='upper left', fontsize=12)

def plot_p_v_p(foo, x, y):
    foo.set_xlabel("Prey Count", fontsize=14)
    foo.set_ylabel("Predator Count", fontsize=14)
    foo.set_title("Predator vs Prey Phase plot", fontsize=16)
    foo.plot(x, y, '-->', markevery=5, label = 'phase plot')
    
def dual_plot(x, y, time):
    fig = plt.figure()
    fig.suptitle(f'Population and Phase Diagrams', fontsize=16)
    plot_pp_v_t(plt.subplot("221"), x, y, time)
    plot_p_v_p(plt.subplot("222"), x, y)

In [119]:
def solve_and_plot(α, β, ɣ, ẟ):
    x, y, time = solve_w_euler(α, β, ɣ, ẟ)
    dual_plot(x, y, time)

Vary the interactions between species by changing $\alpha, \beta, \gamma, \delta$ and see what happens to the population evolution as well as phase plots.

In [120]:
interact(solve_and_plot, α=(0.0,4.0,0.1), β=(0.001,6.0,0.2), ɣ=(1.0,10,1.0), ẟ=(0.0,5.0,0.2));

interactive(children=(FloatSlider(value=2.0, description='α', max=4.0), FloatSlider(value=2.801, description='…

## Further Work: time stepping method with higher order of accuracy

Also, try the same exercise with a fourth order time stepping method called "Runge-Kutta 4" whose algorithm is given below.

In [121]:
def runge(u,f,dt):
    # Runge Kutta 4th order method
    """Returns the solution at the next time-step using Runge Kutta fourth order (RK4) method.
    
    Parameters
    ----------
    u : array of float
        solution at the previous time-step.
    f : function
        function to compute the right hand-side of the system of equation.
    dt : float
        time-increment.
    
    Returns
    -------
    u_n_plus_1 : array of float
        approximate solution at the next time step.
    """
    #calculate slopes
    k1 = f(u)
    u1 = u + (dt/2.)*k1
    k2 = f(u1)
    u2 = u + (dt/2.)*k2
    k3 = f(u2)
    u3 = u + dt*k3
    k4 = f(u3)
    return u + (dt/6.)*(k1 + 2.*k2 + 2.*k3 + k4)